The purpose of this project is to assess how well the suggested passenger traffic model  predicts passenger traffic based on model inputs and in comparison with existing intra-African air passenger traffic values to and from Kenya. The data is contained in an excel file and contains different variables that are assummed to have an effect on air passenger traffic numbers.

In [1]:
import pandas as pd  
import numpy as np  
import matplotlib.pyplot as plt  
import seaborn as sns
import statsmodels.api as sm
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn import metrics
%matplotlib inline

First step is to import the excel workbook and convert it into a Dataframe.

In [22]:
dataset= pd.read_excel(r"C:\Users\user\Documents\Traffic Forecast\TrafficData.xlsx", header=0, delim_whitespace=True)

The column values correspond to the variables that go into the model. The model is based on the following relationship between traffic and the chosen variables: 

𝑇𝑟𝑎𝑓𝑓𝑖𝑐 = 𝐹(𝐺𝐷𝑃,𝐷𝑖𝑠𝑡𝑎𝑛𝑐𝑒,𝐴𝑆𝐴,𝑉𝑖𝑠𝑎 𝑅𝑒𝑔𝑢𝑙𝑎𝑡𝑖𝑜𝑛𝑠,𝐵𝑢𝑠.𝐷𝑒𝑠𝑡.,𝑇𝑜𝑢𝑟.𝐷𝑒𝑠𝑡.,𝐿𝑎𝑛𝑔𝑢𝑎𝑔𝑒) 

This model is expressed in the following log-linear form: 
𝑙𝑛(𝑇𝑟𝑎𝑓𝑓𝑖𝑐) = 𝛽 + 𝛽1𝑙𝑛(𝐺𝐷𝑃) + 𝛽2𝑙𝑛(𝐷𝑖𝑠𝑡𝑎𝑛𝑐𝑒) + 𝛽3 𝐴𝑆𝐴 + 𝛽3 𝑉𝑖𝑠𝑎 𝑅𝑒𝑔𝑢𝑙𝑎𝑡𝑖𝑜𝑛𝑠 + 𝛽5 𝐵𝑢𝑠𝐷𝑒𝑠𝑡 + 𝛽6 𝑇𝑜𝑢𝑟𝐷𝑒𝑠𝑡 + 𝛽7 𝐿𝑎𝑛𝑔𝑢𝑎𝑔𝑒 + 𝜀 

𝑮𝑫𝑷 is the GDP product of the two countries in the country-pair while 𝑑𝑖𝑠𝑡𝑎𝑛𝑐𝑒 is the great Circle distance between the two major airports in the country-pair. 𝑽𝒊𝒔𝒂 𝒓𝒆𝒈𝒖𝒍𝒂𝒕𝒊𝒐𝒏 is a dummy variable that refers to whether citizens of that country need a visa before departure to visit Kenya and whether Kenyan citizens need a visa before departure to visit a specific country. The 𝑨𝑺𝑨 dummy variable captures the liberalization degree for the market provisions Capacity, Designation and Fifth Freedom Rights as well as the overall restrictiveness or liberalness of an ASA. 𝑩𝒖𝒔𝑫𝒆𝒔𝒕 and 𝑻𝒐𝒖𝒓𝑫𝒆𝒔𝒕 are the variables that categorise a country as a tourist or business destination. 𝑳𝒂𝒏𝒈𝒖𝒂𝒈𝒆 is the dummy variable for whether a country has English as one of its official languages or not. 𝜺 is the error term, and 𝒍𝒏 is the natural logarithm. 

In [23]:
dataset.columns.values

array(['Country', 'Timestamp', 'GDP', 'GDP_', 'Distance', 'Traffic',
       'Designation', 'Capacity', 'Rights', 'CoopArr', 'ASA',
       'ForKenyansVisa', 'ToKenyaVisa', 'Language', 'BusDest', 'TourDest'],
      dtype=object)

In [24]:
dataset.shape

(180, 16)

In [25]:
for col in dataset.columns: 
    print(col)

Country
Timestamp
GDP
GDP_
Distance
Traffic
Designation
Capacity
Rights
CoopArr
ASA
ForKenyansVisa
ToKenyaVisa
Language
BusDest
TourDest


The traffic data has been collected over a 5-year period that is from 2014 to 2018 inclusive. In order to capture these years, the timestamp has to be decomposed into its month and year components.

In [26]:
dataset['year'], dataset['month'] = dataset['Timestamp'].dt.year, dataset['Timestamp'].dt.month
dataset

,Country,Timestamp,GDP,GDP_,Distance,Traffic,Designation,Capacity,Rights,CoopArr,ASA,ForKenyansVisa,ToKenyaVisa,Language,BusDest,TourDest,year,month
0,Angola,2014-12-31,51.611232,16.842789,7.921173,9.993831,Restrictive,Restrictive,Restrictive,Restrictive,Restrictive,VN,VNN,No,Yes,No,2014,12
1,Angola,2015-12-31,51.697506,16.869755,7.921173,9.831078,Restrictive,Restrictive,Restrictive,Restrictive,Restrictive,VN,VNN,No,Yes,No,2015,12
2,Angola,2016-12-31,51.750245,16.864499,7.921173,9.323223,Restrictive,Restrictive,Restrictive,Restrictive,Restrictive,VN,VNN,No,Yes,No,2016,12
3,Angola,2017-12-31,51.833910,16.891376,7.921173,10.942031,Restrictive,Restrictive,Restrictive,Restrictive,Restrictive,VN,VNN,No,Yes,No,2017,12
4,Angola,2018-12-31,51.918231,16.919876,7.921173,8.700681,Restrictive,Restrictive,Restrictive,Restrictive,Restrictive,VN,VNN,No,Yes,No,2018,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,Zimbabwe,2014-12-31,49.926687,15.842832,7.573017,10.893716,Liberal,Restrictive,Liberal,Liberal,Restrictive,VNN,VNN,Yes,No,Yes,2014,12
176,Zimbabwe,2015-12-31,50.021212,15.895800,7.573017,10.856785,Liberal,Restrictive,Liberal,Liberal,Restrictive,VNN,VNN,Yes,No,Yes,2015,12
177,Zimbabwe,2016-12-31,50.107620,15.942499,7.573017,10.788267,Liberal,Restrictive,Liberal,Liberal,Restrictive,VNN,VNN,Yes,No,Yes,2016,12
178,Zimbabwe,2017-12-31,50.238727,16.035437,7.573017,10.208985,Liberal,Restrictive,Liberal,Liberal,Restrictive,VNN,VNN,Yes,No,Yes,2017,12


As the data contains observations of different variables over many time periods it is called panel data. Therefore it is necessary to change the dataframe into a multiindex to appropriately capture these observations.

In [27]:
year = pd.Categorical(dataset.year)
dataset = dataset.set_index(['Country', 'year'])
dataset['year'] = year

print(dataset.head())

              Timestamp        GDP       GDP_  Distance    Traffic  \
Country year                                                         
Angola  2014 2014-12-31  51.611232  16.842789  7.921173   9.993831   
        2015 2015-12-31  51.697506  16.869755  7.921173   9.831078   
        2016 2016-12-31  51.750245  16.864499  7.921173   9.323223   
        2017 2017-12-31  51.833910  16.891376  7.921173  10.942031   
        2018 2018-12-31  51.918231  16.919876  7.921173   8.700681   

              Designation     Capacity       Rights      CoopArr          ASA  \
Country year                                                                    
Angola  2014  Restrictive  Restrictive  Restrictive  Restrictive  Restrictive   
        2015  Restrictive  Restrictive  Restrictive  Restrictive  Restrictive   
        2016  Restrictive  Restrictive  Restrictive  Restrictive  Restrictive   
        2017  Restrictive  Restrictive  Restrictive  Restrictive  Restrictive   
        2018  Restricti

The data types for the variables is changed from category to object.

In [29]:
for col in ['Designation', 'Capacity', 'Rights', 'CoopArr', 'ASA', 'ForKenyansVisa', 'ToKenyaVisa', 'Language', 'BusDest', 'TourDest']:
    dataset[col] = dataset[col].astype('object')
    
dataset.dtypes

Timestamp         datetime64[ns]
GDP                      float64
GDP_                     float64
Distance                 float64
Traffic                  float64
Designation               object
Capacity                  object
Rights                    object
CoopArr                   object
ASA                       object
ForKenyansVisa            object
ToKenyaVisa               object
Language                  object
BusDest                   object
TourDest                  object
month                      int64
year                    category
dtype: object

This study shall use dummy variables for the ASA status variable. In the model setup, an ASA is only categorised as liberal if it is labelled as liberal in each of the following market provision: designation, capacity, and fifth freedom rights. A dummy variable is set up for the overall ASA, and the individual designation, capacity, and fifth freedom rights provisions where restrictive ASAs are coded as 1 and liberal ASAs are coded as 0.  

In [30]:
dataset['Designation'].replace({"Restrictive": 1, "Liberal": 0}, inplace=True)

In [31]:
dataset['Capacity'].replace({"Restrictive": 1, "Liberal": 0}, inplace=True)

In [32]:
dataset['Rights'].replace({"Restrictive": 1, "Liberal": 0}, inplace=True)

In [33]:
dataset['CoopArr'].replace({"Restrictive": 1, "Liberal": 0}, inplace=True)

In [34]:
dataset['ASA'].replace({"Restrictive": 1, "Liberal": 0}, inplace=True)

For this study, visa on arrival and no visa shall be grouped together under the category of not needing a visa. A dummy variable is set up where needing a visa is coded as ‘1’ and the no visa grouping is coded as ‘0’.  

dataset['ForKenyansVisa'].replace({"VN": 1, "VNN": 0}, inplace=True)

In [36]:
dataset['ToKenyaVisa'].replace({"VN": 1, "VNN": 0}, inplace=True)

In the model, there is a dummy variable for the language category. Countries that are Englishspeaking are coded as ‘1’ and all other countries are coded as ‘0’. 
In the model, there are dummy variables for business and tourist destinations. For each category, a country is coded ‘1’ if it is a business or tourist destination or both. If the country is not a business nor tourism destination, it will be coded ‘0’ in both categories.

In [37]:
dataset['Language'].replace({"Yes": 1, "No": 0}, inplace=True)

In [38]:
dataset['BusDest'].replace({"Yes": 1, "No": 0}, inplace=True)

In [39]:
dataset['TourDest'].replace({"Yes": 1, "No": 0}, inplace=True)

In [40]:
dataset.head

<bound method NDFrame.head of                Timestamp        GDP       GDP_  Distance    Traffic  \
Country  year                                                         
Angola   2014 2014-12-31  51.611232  16.842789  7.921173   9.993831   
         2015 2015-12-31  51.697506  16.869755  7.921173   9.831078   
         2016 2016-12-31  51.750245  16.864499  7.921173   9.323223   
         2017 2017-12-31  51.833910  16.891376  7.921173  10.942031   
         2018 2018-12-31  51.918231  16.919876  7.921173   8.700681   
...                  ...        ...        ...       ...        ...   
Zimbabwe 2014 2014-12-31  49.926687  15.842832  7.573017  10.893716   
         2015 2015-12-31  50.021212  15.895800  7.573017  10.856785   
         2016 2016-12-31  50.107620  15.942499  7.573017  10.788267   
         2017 2017-12-31  50.238727  16.035437  7.573017  10.208985   
         2018 2018-12-31  50.404382  16.163925  7.573017  11.160328   

               Designation  Capacity  Rights  

From the excel file it is known that there are cells with missing data. As missing data can heavily skew the results of a regression, rows containing missing values need to be removed.

In [42]:
dataset = dataset.dropna()

This study shall perform two regression models. The first model (*Model A*) shall incorporate a single ASA variable which captures the overall liberalness or restrictiveness across the three market provisions, namely capacity, designation and fifth freedom rights. The second model (*Model B*) shall use the dummy variables for the market provisions capacity, designation and fifth freedom rights

In [47]:
from linearmodels.panel import PanelOLS
import statsmodels.api as sm
exog_vars = ['GDP', 'Distance', 'ForKenyansVisa', 'ToKenyaVisa','Language', 'BusDest', 'TourDest', 'ASA']
exog = sm.add_constant(dataset[exog_vars])

modelA = PanelOLS(dataset.Traffic, exog)
modelA_Results = modelA.fit()
print(modelA_Results)

                          PanelOLS Estimation Summary                           
Dep. Variable:                Traffic   R-squared:                        0.4819
Estimator:                   PanelOLS   R-squared (Between):              0.7010
No. Observations:                 146   R-squared (Within):              -0.0034
Date:                Fri, Apr 03 2020   R-squared (Overall):              0.4819
Time:                        16:34:11   Log-likelihood                   -211.52
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      18.338
Entities:                          36   P-value                           0.0000
Avg Obs:                       4.0556   Distribution:                   F(7,138)
Min Obs:                       0.0000                                           
Max Obs:                       5.0000   F-statistic (robust):             18.338
                            

In [46]:
from linearmodels import PanelOLS
import statsmodels.api as sm
exog_vars = ['GDP', 'Distance', 'ForKenyansVisa', 'ToKenyaVisa','Language', 'BusDest', 'TourDest', 'Designation', 'Capacity', 'Rights']
exog = sm.add_constant(dataset[exog_vars])

modelB = PanelOLS(dataset.Traffic, exog)
modelB_Results = modelB.fit()
print(modelB_Results)

                          PanelOLS Estimation Summary                           
Dep. Variable:                Traffic   R-squared:                        0.4992
Estimator:                   PanelOLS   R-squared (Between):              0.7205
No. Observations:                 146   R-squared (Within):              -0.0044
Date:                Fri, Apr 03 2020   R-squared (Overall):              0.4992
Time:                        16:34:07   Log-likelihood                   -209.04
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      15.062
Entities:                          36   P-value                           0.0000
Avg Obs:                       4.0556   Distribution:                   F(9,136)
Min Obs:                       0.0000                                           
Max Obs:                       5.0000   F-statistic (robust):             15.062
                            

A comparison of the two models is done to see how suitable the models are and how the variables coefficients and t-statistics vary. 

R-Squared is a goodness-of-fit measure that shows how well the linear regression is suited to the data. (Lewis-Beck, et al., 2004) The R-Squared measures how well the fit explains the variation in the data. (UNSW Sydney, n.d.) Model A has an R-Squared value of 0.4819. The  R-Squared means that all the variables in the model can explain approximately 48.19% of the passenger traffic. Model B has an R-Squared value of 0.4992. The R-Squared means that all the variables in the model can explain approximately 49.92% of the passenger traffic. 

The F-statistic for models A and B has a p-value of 0.000 which is lower than the p-value of 0.05. Therefore, it is concluded that the results are statistically significant and the effect of the independent variables on the dependent variable is significant.

The values in the brackets contain the significance values for the variables. Even though each variable has a calculated coefficient, not all coefficients are significant. A variable’s coefficient is only significant if the *significance* is less than **0.05**, and the *t-statistic* is more than **+2** and less than **-2**. From the table below, the significant variables for model A are **GDP, Distance, ForKenyansVisa, Language, TourDest** and **ASA**.  The significant variables for model B are **GDP, Distance, ForKenyansVisa, Language, TourDest** and **Rights**. Therefore, the models are mathematically expressed as:

Model A: ln(Traffic)= 0.4416 ln(GDP) - 0.9787 ln(Distance) -0.6578 ASA + 0.7116 ForKenyansVisa -0.5573 Language + 0.6748 TourDest 
 
Model B: ln(Traffic)= 0.4734 ln(GDP) - 1.0055 ln(Distance) + 0.7804 ForKenyansVisa - 0.6091 Language + 0.6273 TourDest - 0.6934 language - 0.698 Rights

In [96]:
from linearmodels.panel import compare
print(compare({'Model A':modelA_Results,'ModelB':modelB_Results}))

                Model Comparison               
                           Model A       ModelB
-----------------------------------------------
Dep. Variable              Traffic      Traffic
Estimator                 PanelOLS     PanelOLS
No. Observations               146          146
Cov. Est.               Unadjusted   Unadjusted
R-squared                   0.4819       0.4992
R-Squared (Within)         -0.0034      -0.0044
R-Squared (Between)         0.7010       0.7205
R-Squared (Overall)         0.4819       0.4992
F-statistic                 18.338       15.062
P-value (F-stat)            0.0000       0.0000
===================== ============ ============
GDP                         0.4416       0.4734
                          (5.9184)     (6.2448)
Distance                   -0.9787      -1.0055
                         (-5.5538)    (-5.4029)
ForKenyansVisa              0.7116       0.7804
                          (3.1783)     (3.3120)
ToKenyaVisa                -4.0759      

In [63]:
x1 = dataset[['GDP', 'Distance', 'ForKenyansVisa', 'ToKenyaVisa','Language', 'BusDest', 'TourDest', 'ASA']]
Y = dataset['Traffic']

trafficpredictionsA = modelA_Results.predict(x1) 

print(trafficpredictionsA)

               predictions
Country  year             
Angola   2014    10.158157
         2015    10.196257
         2016    10.219546
         2017    10.256493
         2018    10.293730
...                    ...
Zimbabwe 2014    10.731610
         2015    10.773353
         2016    10.811511
         2017    10.869408
         2018    10.942563

[146 rows x 1 columns]


In [64]:
x2 = dataset[['GDP', 'Distance', 'ForKenyansVisa', 'ToKenyaVisa','Language', 'BusDest', 'TourDest', 'Designation', 'Capacity', 'Rights']]
Y = dataset['Traffic']

trafficpredictionsB = modelB_Results.predict(x2) 

print(trafficpredictionsB)

               predictions
Country  year             
Angola   2014     9.898464
         2015     9.939304
         2016     9.964269
         2017    10.003875
         2018    10.043790
...                    ...
Zimbabwe 2014    11.114374
         2015    11.159120
         2016    11.200023
         2017    11.262086
         2018    11.340503

[146 rows x 1 columns]


In [77]:
originaltraffic = dataset['Traffic']

print(originaltraffic)

Country   year
Angola    2014     9.993831
          2015     9.831078
          2016     9.323223
          2017    10.942031
          2018     8.700681
                    ...    
Zimbabwe  2014    10.893716
          2015    10.856785
          2016    10.788267
          2017    10.208985
          2018    11.160328
Name: Traffic, Length: 146, dtype: float64


In [89]:
trafficcomparison = pd.concat([originaltraffic, trafficpredictionsA, trafficpredictionsB], axis = 1)


print(trafficcomparison)

                 Traffic  predictions  predictions
Country  year                                     
Angola   2014   9.993831    10.158157     9.898464
         2015   9.831078    10.196257     9.939304
         2016   9.323223    10.219546     9.964269
         2017  10.942031    10.256493    10.003875
         2018   8.700681    10.293730    10.043790
...                  ...          ...          ...
Zimbabwe 2014  10.893716    10.731610    11.114374
         2015  10.856785    10.773353    11.159120
         2016  10.788267    10.811511    11.200023
         2017  10.208985    10.869408    11.262086
         2018  11.160328    10.942563    11.340503

[146 rows x 3 columns]


In the trafficcomparison dataframe below, we can compare how well or how badly the model did when it came to predicting the traffic values for different models. It can be assumed that model B did a better job predicting traffic values due to its higher R-squared value and from the results below it only performs better than model A in some cases.

In [97]:
trafficcomparison.columns = ['Actual Traffic','Model A Prediction','Model B Prediction']

pd.set_option('display.max_rows', None)
print(trafficcomparison)

                               Actual Traffic  Model A Prediction  \
Country                  year                                       
Angola                   2014        9.993831           10.158157   
                         2015        9.831078           10.196257   
                         2016        9.323223           10.219546   
                         2017       10.942031           10.256493   
                         2018        8.700681           10.293730   
Benin                    2014        9.075780            9.703082   
                         2015        7.864036            9.746194   
                         2016        8.382976            9.798197   
                         2017       10.542706            9.860851   
                         2018        8.778326            9.936903   
Burundi                  2014       11.119157            9.784570   
                         2015       10.547838            9.800955   
                         2016     